In [42]:
import pandas as pd

# Questions

## Can we employ advanced sampling techniques to increase the size of our dataset for training?
## What are the most important factors for estimating charter rates
## Are these relationships consistent across vessel types (Capesize, Panamx, etc.)

# Data Cleaning

In [3]:
# read and display the dataset as a pandas DataFrame
df = pd.read_csv('DailyDatabaseFixtures.csv')
df.head()

,ENTRY DATE,WEEK ENDING,ORG CODE,PORT OF ORIGIN,DES CODE,DESTINATION,VESSEL NAME,SIZE,COMM CODE,COMMODITY,DATES,RATE,TERMS,CHARTER
0,10-Oct-19,12-Oct-19,1,Newport News,7.0,Vizagapatnam,STEAMER,75000,1,Coal,Nov 13/22,34.95,FIO;40000tSShex/20000tSShex,SAIL
1,10-Oct-19,12-Oct-19,2,Mobile,5.0,Rotterdam,"ELETTRA, 09",75000,1,Coal,Oct 25/30,16.85,FIO;25000tShinc/25000tShinc,TKSE
2,10-Oct-19,12-Oct-19,8,Dalrymple Bay,7.0,Vizagapatnam,STEAMER,75000,1,Coal,Nov 7/16,18.00,FIO;35000tSShex/20000tSShex,SAIL
3,10-Oct-19,12-Oct-19,4,Santos,9.0,China,STEAMER,65000,2,Hvy Grain,Oct 15/30,32.75,FIO;8000tsShex/8000tSshex,CNR
4,10-Oct-19,12-Oct-19,4,Tubarao,9.0,Qingdao,"SUNBEAM, 00",170000,5,Iron Ore,Oct 12/15,24.25,FIO;3DaysShinc/30000tShinc,Trafigura


Our dataset is small with only 153 rows and many columns containing categorical data types:

In [9]:
df.shape

(153, 14)

We therefore want to avoid dropping rows if possible. With this in mind, let's check for missing vales and see if we can impute sensible values instead of dropping anything.

In [6]:
# check for columns with missing values
df.columns[df.isna().any()]

Index(['DES CODE', 'TERMS'], dtype='object')

In [10]:
# One location has a missing destination code as their destination is listed only as Redel (i.e. redelivery)
df.loc[df['DES CODE'].isna()]

,ENTRY DATE,WEEK ENDING,ORG CODE,PORT OF ORIGIN,DES CODE,DESTINATION,VESSEL NAME,SIZE,COMM CODE,COMMODITY,DATES,RATE,TERMS,CHARTER
19,10-Oct-19,12-Oct-19,4,Del EC SoAmerica,NaN,Redel,"OLYMPIC GALAXY, 09",81383,7,14k/36t,Oct 28/31,18000 Day,3389700GrnBl;TO+$800000Bonus,Fractal


In [23]:
# Taking a look at other possible allocations for this spurious row
df.loc[df['DESTINATION'].str.contains('Redel'), ['DES CODE', 'DESTINATION']].drop_duplicates()

,DES CODE,DESTINATION
11,9.0,Redel China
12,8.0,Redel Philippines
13,5.0,Redel Continent
14,7.0,Redel Sri Lanka
15,9.0,Redel China ViaECIndia
17,9.0,Redel Sing/Japan
18,9.0,Redel So China
19,0.0,Redel Unrptd
23,9.0,Redel Youngheung


In [14]:
# Since there is another destination for redelivery unreported, lets fill this NaN with the same code and destination value as other unreported records:
df.loc[df['DES CODE'].isna(), ['DES CODE', 'DESTINATION']] = df.loc[df['DESTINATION'] == 'Redel Unrptd', ['DES CODE', 'DESTINATION']].values[0]

In [16]:
df.loc[df['VESSEL NAME'] == 'OLYMPIC GALAXY, 09']

,ENTRY DATE,WEEK ENDING,ORG CODE,PORT OF ORIGIN,DES CODE,DESTINATION,VESSEL NAME,SIZE,COMM CODE,COMMODITY,DATES,RATE,TERMS,CHARTER
19,10-Oct-19,12-Oct-19,4,Del EC SoAmerica,0.0,Redel Unrptd,"OLYMPIC GALAXY, 09",81383,7,14k/36t,Oct 28/31,18000 Day,3389700GrnBl;TO+$800000Bonus,Fractal


In [28]:
# Typically in freight, rates are given in World Scale rather than in a specific currency unit.
# Notice that in almost all cases where TERMS in null, rates are either not provided or are in world scale (W) which implies that TERMS provides some additional information to help us standardise rates which are not already in world scale
df.loc[df['TERMS'].isna(), ['RATE', 'TERMS']]

,RATE,TERMS
26,RNR,NaN
32,W130,NaN
36,W155,NaN
37,RNR,NaN
38,W138,NaN
...,...,...
144,RNR,NaN
147,W137.5,NaN
150,RNR,NaN
151,RNR,NaN


In [27]:
# Note that lump sum rates appear as either Lump Sum or LS in TERMS
df.loc[df['TERMS'].notna(), ['RATE', 'TERMS']]

,RATE,TERMS
0,34.95,FIO;40000tSShex/20000tSShex
1,16.85,FIO;25000tShinc/25000tShinc
2,18.00,FIO;35000tSShex/20000tSShex
3,32.75,FIO;8000tsShex/8000tSshex
4,24.25,FIO;3DaysShinc/30000tShinc
...,...,...
143,180000,LS:;OpHongKong$325000
145,410000,Lump Sum
146,RNR,PtC
148,455000,PtC;Lump Sum


In [31]:
# Our column of interest is RATE, so lets remove rows where RATE appears only as RNR (i.e. is withheld)
df = df.loc[df['RATE'] != 'RNR'].reset_index(drop=True)

# All of our data also refers to the same date, so we can drop the ENTRY DATE column
df = df.drop(columns = 'ENTRY DATE')

In [43]:
# We need to make RATE a single unit and convert the datatype to either int or float, appropriately
df.RATE

0       34.95
1       16.85
2       18.00
3       32.75
4       24.25
        ...  
131    410000
132    W137.5
133    455000
134    420000
135      W145
Name: RATE, Length: 136, dtype: object